In [ ]:
# Create a session with retry configuration
def create_session():
    session = requests.Session()

    # Configure retry strategy with exponential backoff
    retries = Retry(
        total=5,  # Total number of retries
        backoff_factor=1,  # Exponential backoff factor
        status_forcelist=[429, 500, 502, 503, 504],  # Status codes to retry on
        allowed_methods=["POST", "GET"],  # HTTP methods to retry
    )

    # Apply retry strategy to both http and https
    adapter = HTTPAdapter(max_retries=retries, pool_connections=10, pool_maxsize=20)
    session.mount("http://", adapter)
    session.mount("https://", adapter)

    return session

def search_serper(query, api_key, retries=3):
    """
    Search using Serper API for the specific query
    search_type can be "search" or "maps"
    """
    # Determine the endpoint based on the search type
    endpoint = "https://google.serper.dev/search"

    headers = {
        "X-API-KEY": api_key,
        "Content-Type": "application/json"
    }

    search_params = {
        "q": query,
        "gl": "us",
        "hl": "en"
    }

    # we want to limit results
    search_params["num"] = 6

    # Create a dedicated session for this request
    session = create_session()

    try:
        # Add randomized delay to avoid hitting rate limits and predictable patterns
        jitter = random.uniform(MIN_REQUEST_DELAY, MAX_REQUEST_DELAY)
        time.sleep(jitter)

        response = session.post(endpoint, headers=headers, json=search_params, timeout=(10, 30))
        response.raise_for_status()
        # print(response.json())
        return response.json()

    except Exception as e:
        print(f"Error searching for '{query}': {e}")
        # If we still have retries left and it's a connection error, try again with longer delay
        if retries > 0 and ("Connection" in str(e) or "Timeout" in str(e) or "Max retries" in str(e)):
            print(f"Retrying search for '{query}' ({retries} retries left)")
            # Exponential backoff
            time.sleep(2 ** (4 - retries) + random.uniform(1, 3))
            return search_serper(query, api_key, retries - 1)
        return None
    finally:
        # Close the session to release resources
        session.close()

def process_agency(product_name):
    """
    Process a single product name:
    Search for "product name ingredients list"
    """
    try:
        # Search for product name + "ingredients list"
        website_query = f"{product_name}"
        website_results = search_serper(website_query, API_KEY)
        return website_results

    except Exception as e:
        print(f"Error processing {product_name}: {e}")


def get_webpage_url(user_query):
    # website_results = process_agency(user_query)
    website_results = process_agency(user_query)
    if "organic" in website_results and len(website_results["organic"]) > 0:
        for each_result in website_results["organic"]:
            if "incidecoder.com" in each_result.get("link", "") :
                return each_result.get("link", "")
        return website_results["organic"][0].get("link", "")
    return ""